In [0]:
dbutils.secrets.listScopes()

In [0]:
dbutils.secrets.list('gijoformula1-scope')

In [0]:
%fs ls dbfs:/FileStore/tables/

In [0]:
%pip install azure-eventhub azure-identity

In [0]:
%restart_python

In [0]:
eventhub_secret = dbutils.secrets.get(scope = 'gijoformula1-scope', key='keh-client-secret')

In [0]:
display(eventhub_secret)

In [0]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

In [0]:

# --- Parameters ---
bootstrap = "creditrisk.servicebus.windows.net:9093"   # EH Kafka endpoint
event_hub = "transactions"                              # your Event Hub (topic)
eh_conn   = dbutils.secrets.get(scope = 'gijoformula1-scope', key='keh-client-secret')  # full EH connection string


In [0]:
# --- Read stream from Event Hubs via Kafka protocol ---
raw = (spark.readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", bootstrap)
  .option("subscribe", event_hub)
  .option("startingOffsets", "latest")
  .option("kafka.security.protocol", "SASL_SSL")
  .option("kafka.sasl.mechanism", "PLAIN")
  .option("kafka.sasl.jaas.config",
          f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{eh_conn}";')
  .option("kafka.client.dns.lookup", "use_all_dns_ips")      # recommended for EH
  .option("kafka.request.timeout.ms", "60000")               # optional
  .load())


In [0]:

# --- (Example) Parse JSON payload to columns ---
schema = StructType([
  StructField("tx_id", StringType()),
  StructField("user_id", StringType()),
  StructField("amount", DoubleType()),
  StructField("ts", TimestampType())
])


In [0]:
events = (raw
  .select(from_json(col("value").cast("string"), schema).alias("data"))
  .select("data.*"))

In [0]:
display(events)

In [0]:
# --- Write to Delta with checkpoint (exactly-once) ---
(events.writeStream
  .format("delta")
  .option("checkpointLocation", "dbfs:/chk/eh/transactions")
  .outputMode("append")
  .toTable("lakehouse.silver.transactions"))
